In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/kaggle/input/essays/data.csv', index_col=0)

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [5]:
from datasets import Dataset

In [6]:
class EllipseDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.df = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = row["full_text"].values.tolist()

        scores = torch.tensor(row[['Cohesion',
                                   'Syntax',
                                   'Vocabulary',
                                   'Phraseology',
                                   'Grammar',
                                   'Conventions']
                                  ].values, dtype=torch.float)

        labels = torch.tensor(row[["Cohesion_normalized",
                                   "Syntax_normalized",
                                   "Vocabulary_normalized",
                                   "Phraseology_normalized",
                                   "Grammar_normalized",
                                   "Conventions_normalized"]
                                  ].values, dtype=torch.float)

        inputs = self.tokenizer(text, padding="max_length", max_length=64, truncation=True, return_tensors="pt")

        return {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "labels": labels,
            "scores": scores
        }

In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model = AutoModelForSequenceClassification.from_pretrained("Kevintu/Engessay_grading_ML")
tokenizer = AutoTokenizer.from_pretrained("KevSun/Engessay_grading_ML")

2025-04-12 19:39:25.951144: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744486765.991290     323 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744486766.003428     323 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
from torch.utils.data import DataLoader
import torch.optim as optim

In [9]:
test_dataset = EllipseDataset(test_df.reset_index(drop=True), tokenizer)

In [10]:
test_loader = DataLoader(test_dataset, batch_size=16)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=4e-5)

In [11]:
import numpy as np

In [12]:
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["scores"].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        preds = outputs.logits.squeeze()
        predicted_scores = preds.cpu().numpy()
        predictions.extend(predicted_scores)
        true_labels.extend(labels.cpu().numpy())

In [13]:
predictions_new = np.array(predictions[-6:])

In [14]:
predictions_new1 = predictions[:-6]
predictions_new1.append(predictions_new)

In [15]:
from sklearn.metrics import cohen_kappa_score

In [ ]:
criteria_true

In [ ]:
y_true_original

In [16]:
y_min = 1
y_max = 5
for i, criteria in enumerate(['Cohesion', 'Syntax', 'Vocabulary', 'Phraseology', 'Grammar', 'Conventions']):
    criteria_pred = np.array([arr[i] for arr in predictions_new1], dtype=np.float32)
    criteria_true = np.array([arr[i] for arr in true_labels], dtype=np.float32)
    # criteria_true_original = criteria_true * (y_max - y_min) + y_min
    y_pred_original = (np.round(criteria_pred * 2) - 2).astype(int)
    y_true_original = ((criteria_true - 1) * 2).astype(int)
    score = cohen_kappa_score(y_true_original, y_pred_original, weights='quadratic')
    print(f'QWK_{criteria} = {score}')

QWK_Cohesion = 0.5395658486208519
QWK_Syntax = 0.5967416064243323
QWK_Vocabulary = 0.5388103427116735
QWK_Phraseology = 0.5833546224428662
QWK_Grammar = 0.5718992865664829
QWK_Conventions = 0.5810171161591173


In [ ]:
tokenizer